In [ ]:
# Імпорт необхідних бібліотек та класів
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise import SVD, SVDpp, NMF
import random
import numpy as np

# Задайте шлях до CSV файлів датасету Movielens
data_path = r'C:\Users\Сергій\Documents\Code\Current_task\Math-test\task7'

# Завантажте дані з файлу ratings.csv та визначте формат даних
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file(data_path + r'\ratings.csv', reader=reader)

# Зменшимо розмір даних, вибравши випадковий піднабір
random.seed(42)  # Зафіксуємо seed для відтворюваності
sampled_data = random.sample(data.raw_ratings, k=int(0.1 * len(data.raw_ratings)))  # Використовуйте 10% даних

# Завантажте піднабір даних
data.raw_ratings = sampled_data

# Визначте список алгоритмів, які ви хочете спробувати (SVD, SVD++, NMF)
algo_candidates = [
    SVD(),
    SVDpp(),
    NMF(),
]

# Зменшимо кількість значень для n_epochs і n_factors
param_grid = {
    'n_epochs': [5, 10],  # Зменшено кількість значень
    'n_factors': [50, 100],  # Зменшено кількість значень
}

# Ітеруйтесь через алгоритми та знаходьте найкращі параметри
best_params = {}
best_score = 0

for algo in algo_candidates:
    grid_search = GridSearchCV(algo, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
    grid_search.fit(data)
    
    # Отримайте найкращий алгоритм та його параметри
    best_algo = grid_search.best_estimator['rmse']
    best_params[algo.__class__.__name__] = grid_search.best_params['rmse']
    
    # Отримайте кращий RMSE
    rmse = grid_search.best_score['rmse']
    if rmse < best_score or best_score == 0:
        best_score = rmse
        best_algorithm = algo.__class__.__name__

# Виведіть результати
print(f"Найкращий алгоритм: {best_algorithm}")
print(f"Найкращі параметри: {best_params[best_algorithm]}")
print(f"Кращий RMSE: {best_score}")

# Параметри моделі
n_users = len(data.raw_ratings)
n_items = len(set([item for (_, item, _) in data.raw_ratings]))
n_factors = 100
learning_rate = 0.01
reg_param = 0.02
n_epochs = 10

# Створюємо та навчаємо модель
model = MatrixFactorization(n_users, n_items, n_factors, learning_rate, reg_param, n_epochs)
train_data = [(user, item, rating) for (user, item, rating, _) in data.raw_ratings]
model.train(train_data)

# Приклад передбачення
user_id = 0
item_id = 1
predicted_rating = model.predict(user_id, item_id)
print(f"Прогнозований рейтинг для користувача {user_id} та предмету {item_id}: {predicted_rating}")
